In [1]:
import time # Want VAE fitting run time comparison

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from statistics import mean
from statistics import stdev

import glob as glob

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization, Activation, Layer
from tensorflow.keras import metrics, optimizers
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import Callback

import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import umap.umap_ as umap

In [2]:
# Latent compute
def compute_latent(x): # x:
    mu, sigma = x
    batch = K.shape(mu)[0]
    dim = K.shape(mu)[1]
    eps = K.random_normal(shape=(batch,dim), mean=0., stddev=1.0 )
    return mu + K.exp(sigma/2)*eps

In [3]:
# Custom layer for loss
class CustomVariationalLayer(Layer):
    """
    Define a custom layer
    """
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

In [4]:
class WarmUpCallback(Callback):
    def __init__(self, beta, kappa):
        self.beta = beta
        self.kappa = kappa

    def on_epoch_end(self, epoch, logs={}):
        if K.get_value(self.beta) <= 1:
            K.set_value(self.beta, K.get_value(self.beta) + self.kappa)

In [5]:
train_paths_5k = sorted(glob.glob('data_5k/*.tsv'))

In [7]:
train_paths_5k = sorted(glob.glob('data_5k/*.tsv'))
for train_path in train_paths_5k:
    train_df = pd.read_csv(train_path, sep="\t", index_col=0)
    train_cohort = train_df.index.name
    features = train_df.columns[1:]
    
    original_dim = len(features)
    feature_dim = len(features)
    latent_dim = 100
    
    encoder_inputs = keras.Input(shape=(feature_dim,))
    z_mean_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_1")(encoder_inputs)
    z_mean_dense_batchnorm = layers.BatchNormalization()(z_mean_dense_linear)
    z_mean_encoded = layers.Activation('relu')(z_mean_dense_batchnorm)

    z_log_var_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_2")(encoder_inputs)
    z_log_var_dense_batchnorm = layers.BatchNormalization()(z_log_var_dense_linear)
    z_log_var_encoded = layers.Activation('relu')(z_log_var_dense_batchnorm)

    latent_space = layers.Lambda(compute_latent, output_shape=(latent_dim,), name="latent_space")([z_mean_encoded, z_log_var_encoded])

    decoder_to_reconstruct = layers.Dense(feature_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
    decoder_outputs = decoder_to_reconstruct(latent_space)

    learning_rate = 0.0005
    kappa = 1

    beta = K.variable(0)

    adam = optimizers.Adam(learning_rate=learning_rate)
    vae_layer = CustomVariationalLayer()([encoder_inputs, decoder_outputs])
    vae = Model(encoder_inputs, vae_layer)
    vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

    # X_train = train_df.iloc[:, 1:]

    epochs=100

    fit_start = time.time()
    history = vae.fit(train_df.iloc[:, 1:],  
                epochs=epochs, batch_size=50, shuffle=True,
                callbacks=[WarmUpCallback(beta, kappa)],
                     verbose=0)
    fit_end = time.time() - fit_start

    plt.plot(history.history['loss'],label="loss")

    plt.title(
        train_cohort+' loss for direct predict on other cohorts n = 24'+
        '\nTybalt VAE train\n'
             )
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.annotate('Feature set = '+'MAD 5000k'+
                 '\nLatent dim = '+str(latent_dim)+
                 '\nLayer type = dense'+
                 '\nNormalization = divide by max()',
                xy=(.4, .8), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                )

    plt.legend(loc="lower left")
    plt.savefig(
        'RNB00978_runtimes/'+train_cohort+'_loss_'+
            str(round(fit_end,2))+'_runtime.png',
        bbox_inches='tight')
    plt.close()

Instructions for updating:
Colocations handled automatically by placer.


2022-11-07 14:07:51.301975: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
ls

Casey_Greene_baseline.ipynb
Classical ML on raw vs decoded notes.rtf
Code_demo_2022-11-02.rtf
J-Lab_Exa_blog.txt
RNB00978_runtimes/
a_ntrsct_00.ipynb
b_model_trn_one_prdct_anoth_00.ipynb
baseline_00.ipynb
baseline_01.ipynb
baseline_02.ipynb
baseline_03.ipynb
baseline_04.ipynb
baseline_05.ipynb
clf_org_dcd_ovrly.ipynb
d_org-dcd.ipynb
data/
data_5k/
decoded/
decoded_baseline_F1s/
decoded_baseline_plots/
mad5k.ipynb
matrices_build.ipynb
non-RNA.ipynb
normalizer.ipynb
original_normalized/
overlay_plots/
runtime_check-Copy1.ipynb
transfer_learning_loss_plots/
umap.ipynb
